In [1]:
import pickle
import numpy as np
import neat
import datetime
import os
import math

from comunication_channel import AgentLogChannel

from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.base_env import ActionTuple
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel

In [2]:
NUM_RUNS = 1
MAX_GENS = 3000

generation = 0
SAVE_INTERVAL = 50

In [3]:
from pureples.shared.visualize import draw_net
from pureples.shared.substrate import Substrate
from pureples.hyperneat.hyperneat import create_phenotype_network
INPUT_COORDINATES = []
for y in range(8):  # 8 riadkov
    for x in range(10):  # 10 stĺpcov
        if len(INPUT_COORDINATES) < 76:  # Pridáme len prvých 76
            INPUT_COORDINATES.append((-1. + (2.*x/9.), -1. + (2.*y/7.)))

# Vytvorenie výstupných súradníc pre 4 výstupy v jednom riadku
OUTPUT_COORDINATES = [(-0.75, 1.), (-0.25, 1.), (0.25, 1.), (0.75, 1.)]

# Vytvorenie substrátu
SUBSTRATE = Substrate(
    INPUT_COORDINATES, OUTPUT_COORDINATES)

In [4]:
engine_config_channel = EngineConfigurationChannel()
engine_config_channel.set_configuration_parameters(time_scale=4)
agent_count_channel = AgentLogChannel()

env_path = "../Builds/train-env/autonomous-drone.exe"
save_nn_destination = 'result/best.pkl'

env = UnityEnvironment(file_name=None, worker_id=0, no_graphics=True, side_channels=[engine_config_channel, agent_count_channel])
env.reset()
env.reset()

In [5]:

behavior_specs = env.behavior_specs
print(f"Behaviour specs {behavior_specs}")
behavior_name = list(behavior_specs)[0]
spec = env.behavior_specs[behavior_name]

print(f"Name of the behavior : {behavior_name}")
print("Number of observations : ", len(spec.observation_specs)) # vector if 1

# Is the Action continuous or multi-discrete ?
if spec.action_spec.continuous_size > 0:
  print(f"There are {spec.action_spec.continuous_size} continuous actions")
if spec.action_spec.is_discrete():
  print(f"There are {spec.action_spec.discrete_size} discrete actions")

Behaviour specs <mlagents_envs.base_env.BehaviorMapping object at 0x000001A5E1C9B5E0>
Name of the behavior : ForestControl?team=0
Number of observations :  5
There are 4 continuous actions


In [6]:
def create_folder(file_name_prefix):
    directory = os.path.dirname(f"{file_name_prefix}")
    if not os.path.exists(directory):
        os.makedirs(directory)

In [7]:
def set_agents_and_double_reset(num_agents: int):
    agent_count_channel.send_int(data=num_agents) 
    env.reset()
    env.reset()

In [8]:
def get_observation_for_agent(agent: int, observations):
    for observation in observations:
        key = int(observation.split("=")[2])
        if key == agent:
            return observations[observation]

In [9]:
def map_agent_ids(decision_steps):
    """
    Map agent ids between NEAT and UNITY.

    Args:
        decision_steps: An iterable containing decision steps.

    Returns:
        A tuple of two dictionaries: (unity_to_neat_map, neat_to_unity_map)
    """
    unity_to_neat_map = {}
    neat_to_unity_map = {}
    id_count = 0
    for step in decision_steps:
        unity_to_neat_map[step] = id_count
        neat_to_unity_map[id_count] = step
        id_count += 1
    return unity_to_neat_map, neat_to_unity_map

In [10]:
def create_policies(genomes, cfg):
    nets = []
    for _, g in genomes:
        g.fitness = 0
        policy = neat.nn.FeedForwardNetwork.create(g, cfg)
        net = create_phenotype_network(policy, SUBSTRATE)
        nets.append(net)
    return nets

In [11]:
def eval_genomes(genomes, cfg):
    global generation 
    generation += 1

    policies = create_policies(genomes, cfg)
    set_agents_and_double_reset(len(policies))
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    agent_count = len(decision_steps.agent_id)

    unity_to_neat_map, neat_to_unity_map = map_agent_ids(decision_steps)

    done = False  # Vectorized initialization
    removed_agents = []

    episode_rewards = [0] * agent_count
    print(f"Agent count: {agent_count}")

    while not done:
        for agent in decision_steps:
            if unity_to_neat_map[agent] not in removed_agents:
                obs = np.concatenate(decision_steps[agent].obs[:])
                nn_input =  np.asarray(obs)
                #print(f"NN INPUT: {nn_input}, agent: {agent}")
                #print(f"NN INPUT[0]: {nn_input[0]}, agent: {agent}")
                actions = policies[unity_to_neat_map[agent]].activate(nn_input)
                #print(f"NN OUTPUT: {actions}, agent: {agent}")
                continous_actions = np.asarray([actions])
                continous_actions = np.clip(continous_actions, -1, 1)
                action_tuple = ActionTuple(discrete=None, continuous=continous_actions)
                env.set_action_for_agent(behavior_name=behavior_name, 
                                        agent_id=agent, 
                                        action=action_tuple)
        env.step()
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        for agent in range(agent_count):
            if agent not in removed_agents:
                local_agent = neat_to_unity_map[agent]
                if local_agent in terminal_steps:
                    episode_rewards[agent] += terminal_steps[local_agent].reward
                    removed_agents.append(agent)
                    #print(f"Finished: {agent}")
                elif local_agent in decision_steps:
                    episode_rewards[agent] += decision_steps[local_agent].reward
                 

        if len(removed_agents) >= agent_count:
            done = True
    for i, (_, genome) in enumerate(genomes):
        genome.fitness = episode_rewards[i]
    if generation % SAVE_INTERVAL == 0:
      global file_name_prefix
      best_genome_current_generation = max(genomes, key=lambda x: x[1].fitness) 
      with open(file_name_prefix+'best-'+str(generation)+'.pkl', 'wb') as f:
          pickle.dump(best_genome_current_generation, f)
      

In [12]:
def run(config_file, run, datte):
    print(f"Running {run}")
    global file_name_prefix
    file_name_prefix = f"checkpoints/{datte}/run-{run}/"
    create_folder(file_name_prefix=file_name_prefix)

    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)
    
    pop = neat.Population(config)

    stats = neat.StatisticsReporter()

    pop.add_reporter(stats)
    pop.add_reporter(neat.TBReporter(True, 0, run, datte))
    #pop.add_reporter(neat.StdOutReporter(True))
    best = pop.run(eval_genomes, MAX_GENS)
    # Display the winning genome.
    # print('\nBest genome:\n{!s}'.format(best))
    print("Finished running!")
    
    # Save best genome
    with open(f'logs/{datte}/{run}/best.pkl', 'wb') as f:
        pickle.dump(best, f)

In [13]:
config_path = '../configs/config-feedforward-forest'
datte = datetime.datetime.now().strftime("%d-%m-%Y--%H_%M")
# pre create folders for checkpointer
for r in range(NUM_RUNS):
    run(config_path, r, datte)

Running 0

 ****** Running generation 0 ****** 

Agent count: 50
Population's average fitness: -1.00000 stdev: 0.00000
Best fitness: -1.00000 - size: (4, 152) - species 1 - id 1
Best fitness ever found! : -1.0
Population of 50 members in 2 species:
   ID   age  size   fitness   adj fit  stag
  ====  ===  ====  =========  =======  ====
     1    0    36       -1.0      0.0     0
     2    0    14          0        0     0
Total extinctions: 0
Generation time: 24.888 sec

 ****** Running generation 1 ****** 

Agent count: 50
Population's average fitness: -1.00000 stdev: 0.00000
Best fitness: -1.00000 - size: (4, 152) - species 1 - id 1
Population of 50 members in 2 species:
   ID   age  size   fitness   adj fit  stag
  ====  ===  ====  =========  =======  ====
     1    1    25       -1.0      0.0     1
     2    1    25       -1.0      0.0     0
Total extinctions: 0
Generation time: 24.107 sec (24.498 average)

 ****** Running generation 2 ****** 

Agent count: 50
Population's average f

In [ ]:
env.close()